In [166]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor

In [167]:
#https://www.kaggle.com/datasets/ramprasad273/predicting-food-delivery-time
df = pd.read_excel("Participants Data/Data_Train.xlsx")

In [168]:
df = df.rename(columns={"Delivery_Time": "Preparation_Time"})

In [169]:
df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Preparation_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [170]:
df["Average_Cost"].unique()

array(['₹200', '₹100', '₹150', '₹250', '₹650', '₹350', '₹800', '₹50',
       '₹400', '₹600', '₹300', '₹750', '₹450', '₹550', '₹1,000', '₹500',
       '₹900', '₹1,200', '₹950', '₹850', '₹700', '₹1,150', 'for',
       '₹1,100', '₹1,400', '₹2,050'], dtype=object)

In [171]:
df.drop(df[df["Average_Cost"] == "for"].index, inplace=True)

In [172]:
df["Average_Cost"].value_counts()

₹200      3241
₹100      2557
₹150      2462
₹250       881
₹300       537
₹350       283
₹400       282
₹50        265
₹600       154
₹500       101
₹450        63
₹550        60
₹650        55
₹800        44
₹750        38
₹900        15
₹700        15
₹1,000      12
₹850        12
₹1,200       8
₹950         4
₹1,150       1
₹1,100       1
₹1,400       1
₹2,050       1
Name: Average_Cost, dtype: int64

In [173]:
df["Preparation_Time"] = df["Preparation_Time"].apply(lambda x: int(x.split()[0]))


In [174]:
df["Average_Cost"] = df["Average_Cost"].apply(lambda x: int(x[1:].replace(",","")))

In [175]:
df["Minimum_Order"] = df["Minimum_Order"].apply(lambda x: int(x[1:]))

In [176]:
df.shape

(11093, 9)

In [180]:
tmp = []
for i in range(df.shape[0]):
    if df["Rating"].iloc[i] == "-":
        continue
    elif df["Rating"].iloc[i] == "NEW":
        continue
    elif df["Rating"].iloc[i] == "Opening Soon":
        continue
    elif df["Rating"].iloc[i] == "Temporarily Closed":
        continue
    else:
        tmp.append(df["Rating"].iloc[i])

In [181]:
avg_rate = np.array(tmp).astype(float).mean()
avg_rate

3.6134596429744827

In [182]:
df["Rating"] = df["Rating"].apply(lambda x: avg_rate if x in ["NEW", "-", "Opening Soon", "Temporarily Closed"] else x).astype(float)


In [184]:
from hashlib import sha256

In [185]:
s = "FTI College, Law College Road, Pune"
int(sha256(s.encode()).hexdigest(), 16) % (10 ** 8)

16347948

In [186]:
def str2int(s):
    return int(sha256(s.encode()).hexdigest(), 16) % (10 ** 8)

In [187]:
df["Votes"] = df["Votes"].apply(lambda x: float(x) if x != "-" else 0)
df["Reviews"] = df["Reviews"].apply(lambda x: float(x) if x != "-" else 0)

In [188]:
df["Location"] = df["Location"].apply(str2int)

In [189]:
df["Cuisines"] = df["Cuisines"].apply(str2int)

In [190]:
df.columns

Index(['Restaurant', 'Location', 'Cuisines', 'Average_Cost', 'Minimum_Order',
       'Rating', 'Votes', 'Reviews', 'Preparation_Time'],
      dtype='object')

In [211]:
df

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Preparation_Time
0,ID_6321,16347948,68420766,200,50,3.50000,12.0,4.0,30
1,ID_2882,9753282,4745044,100,50,3.50000,11.0,4.0,30
2,ID_1595,87454491,25397617,150,50,3.60000,99.0,30.0,65
3,ID_5929,33210479,44047920,250,99,3.70000,176.0,95.0,30
4,ID_6123,64748422,67762926,200,99,3.20000,521.0,235.0,65
...,...,...,...,...,...,...,...,...,...
11089,ID_8067,39333219,15885595,250,50,4.20000,326.0,189.0,30
11090,ID_4620,37963703,46139923,100,50,3.60000,36.0,16.0,30
11091,ID_3392,10584479,55298936,100,50,3.50000,45.0,18.0,30
11092,ID_4115,9753282,59943232,100,50,3.10000,24.0,9.0,30


In [212]:
train_X = df[["Location", "Cuisines", "Average_Cost", "Minimum_Order", "Rating", "Votes", "Reviews"]].values

In [216]:
train_y = df["Preparation_Time"].values

In [217]:
reg = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=0)

In [219]:
reg.fit(train_X, train_y)

RandomForestRegressor(max_depth=7, n_estimators=1000, random_state=0)

In [220]:
reg.score(train_X, train_y)

0.403090677116975

In [225]:
import sklearn
from sklearn import ensemble

In [224]:
# Import Random forest architecture model from sklearn
from sklearn.ensemble import RandomForestRegressor
# Construct a random forest regressor
reg = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=0)
# Fit the model to the training data
reg.fit(train_X, train_y)
# score the model (R^2 score)
score = reg.score(train_X, train_y)
print("Score for Random Forest Regressor: ", score)

Score for Random Forest Regressor:  0.403090677116975


In [227]:
# Import Random forest architecture model from sklearn
from sklearn.ensemble import GradientBoostingRegressor
# Construct a random forest regressor
reg = GradientBoostingRegressor(n_estimators=1000, max_depth=7, random_state=0)
# Fit the model to the training data
reg.fit(train_X, train_y)
# score the model (R^2 score)
score = reg.score(train_X, train_y)
print("Score for Random Forest Regressor: ", score)


Score for Random Forest Regressor:  0.973051704897379


In [232]:
from sklearn.model_selection import cross_validate

In [233]:
cv_results = cross_validate(reg, train_X, train_y, cv=5, scoring=('neg_mean_absolute_error', 'neg_mean_squared_error', 'r2'), return_train_score=True)

In [234]:
pd.DataFrame(cv_results)

,fit_time,score_time,test_neg_mean_absolute_error,train_neg_mean_absolute_error,test_neg_mean_squared_error,train_neg_mean_squared_error,test_r2,train_r2
0,13.116164,0.046000,-5.746036,-0.854429,-92.276210,-3.403603,0.342901,0.978533
1,13.203513,0.047042,-6.018684,-0.791807,-98.237235,-2.639340,0.346495,0.983090
2,13.204349,0.051989,-5.582568,-0.810261,-90.936913,-3.089812,0.390515,0.980239
3,12.825132,0.045021,-6.125243,-0.826135,-105.443386,-3.235228,0.339831,0.978956
4,12.873403,0.045014,-6.166170,-0.799717,-117.594330,-2.575306,0.327597,0.982825


In [196]:
pred = reg.predict(train_X)

In [197]:
pred

array([31.54403739, 32.22286188, 39.17678343, ..., 38.197556  ,
       35.22797027, 30.8793661 ])

In [221]:
train_y

array([30, 30, 65, ..., 30, 30, 30], dtype=int64)

In [222]:
pd.DataFrame({"predict": pred, "actual": train_y}).sample(5)

,predict,actual
3189,35.234073,45
1802,32.698875,30
6758,32.301681,30
689,43.341962,30
3290,32.538438,30


In [223]:
pd.DataFrame({"predict": pred, "actual": train_y}).sample(10)

,predict,actual
10284,36.996291,30
3631,31.486439,30
2868,33.599868,30
1415,30.909826,30
2294,42.001141,45
6266,35.450548,30
7901,42.665936,45
6365,48.571634,65
2179,31.618625,30
5296,30.815508,30


In [199]:
df_without_target = df.iloc[:,1:-1]
df_without_target.head()

,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,16347948,68420766,200,50,3.5,12.0,4.0
1,9753282,4745044,100,50,3.5,11.0,4.0
2,87454491,25397617,150,50,3.6,99.0,30.0
3,33210479,44047920,250,99,3.7,176.0,95.0
4,64748422,67762926,200,99,3.2,521.0,235.0


In [200]:
import time
import numpy as np

In [201]:
n = 100
durations = []
for i in range(n):
    start_time = time.time()
    sample_data = df_without_target.sample(1)
    result = reg.predict(sample_data)
    end_time = time.time()
    duration = end_time - start_time
    durations.append(duration)
duration_mean = np.mean(durations)
duration_median = np.median(durations)
duration_std =  np.std(durations)
print("Mean duration: {:.2f} seconds".format(duration_mean))
print("Median duration: {:.2f} seconds".format(duration_median))
print("Standard deviation: {:.2f} seconds".format(duration_std))

Mean duration: 0.06 seconds
Median duration: 0.05 seconds
Standard deviation: 0.01 seconds


In [ ]:
n = 100
durations = []
for i in range(n):
    start_time = time.time()
    sample_data = df_without_target.sample(1)
    result = reg.predict(sample_data)
    end_time = time.time()
    duration = end_time - start_time
    durations.append(duration)
duration_mean = np.mean(durations)
duration_median = np.median(durations)
duration_std =  np.std(durations)
print("Mean duration: {:.2f} seconds".format(duration_mean))
print("Median duration: {:.2f} seconds".format(duration_median))
print("Standard deviation: {:.2f} seconds".format(duration_std))

In [236]:
df.shape

(11093, 9)